# Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from pathlib import Path
import yaml
import edurel.utils.duck_utils as ddbu

BASE_DIR = "/home/basis/work/github/edurel"
DB_DIR = f"{BASE_DIR}/databases/"



# Misc

In [3]:
p = Path(BASE_DIR) / "tmp/adw-oltp/csv"
p

PosixPath('/home/basis/work/github/edurel/tmp/adw-oltp/csv')

In [4]:
files = sorted([f.name for f in p.iterdir() if f.is_file()])
print(files)

['BusinessEntity.csv', 'BusinessEntityAddress.csv', 'BusinessEntityContact.csv', 'EmailAddress.csv', 'Password.csv', 'Person.csv', 'PersonPhone.csv', 'PhoneNumberType.csv', 'ProductModel.csv', 'Store.csv']


In [5]:
tns = [Path(f).stem for f in files]
tns

['BusinessEntity',
 'BusinessEntityAddress',
 'BusinessEntityContact',
 'EmailAddress',
 'Password',
 'Person',
 'PersonPhone',
 'PhoneNumberType',
 'ProductModel',
 'Store']

In [25]:
con = ddbu.mem_con()
ddbu.sql_print(con, f"select * from duckdb_constraints()")

┌───────────────┬──────────────┬─────────────┬────────────┬────────────┬───────────┬──────────────────┬─────────────────┬─────────────────┬────────────┬───────────────────────────┬─────────────────────────┬─────────────────┬──────────────────┬─────────────────────────┐
│ database_name │ database_oid │ schema_name │ schema_oid │ table_name │ table_oid │ constraint_index │ constraint_type │ constraint_text │ expression │ constraint_column_indexes │ constraint_column_names │ constraint_name │ referenced_table │ referenced_column_names │
│    varchar    │    int64     │   varchar   │   int64    │  varchar   │   int64   │      int64       │     varchar     │     varchar     │  varchar   │          int64[]          │        varchar[]        │     varchar     │     varchar      │        varchar[]        │
├───────────────┴──────────────┴─────────────┴────────────┴────────────┴───────────┴──────────────────┴─────────────────┴─────────────────┴────────────┴───────────────────────────┴──────────

# FileDB

In [ ]:
con = ddbu.file_con(f"{DB_DIR}/db-adw-oltp", read_only=True)

In [ ]:
con.close()

# MemDB

In [27]:
con = ddbu.mem_con(DB_DIR + "db-hvs")
print(ddbu.schema(con))

# con.close()

Table: belegung (kursbt_kbtid INTEGER NOT NULL, studentin_prid INTEGER NOT NULL)
Table: einzeltermin (tid INTEGER NOT NULL, datumuhrzeitvon DATE NOT NULL, datumuhrzeitbis DATE NOT NULL)
Table: kurs (kid INTEGER NOT NULL, modul_mid INTEGER NOT NULL, semester_sid INTEGER NOT NULL)
Table: kursbt (kbtid INTEGER NOT NULL, kurs_kid INTEGER NOT NULL, modulbt_mbtid INTEGER NOT NULL, raum_rid INTEGER NOT NULL, lehrperson_prid INTEGER NOT NULL, termin_tid INTEGER NOT NULL)
Table: lehrperson (prid INTEGER NOT NULL, datumbeginn DATE NOT NULL, datumende DATE NULL, pid INTEGER NOT NULL, rid INTEGER NOT NULL, steuernummer VARCHAR NOT NULL)
Table: modul (mid INTEGER NOT NULL, bez VARCHAR NOT NULL, cp INTEGER NOT NULL, regelsemester INTEGER NOT NULL, modulart_maid INTEGER NOT NULL)
Table: modulart (maid INTEGER NOT NULL, bez VARCHAR NOT NULL, aktiv VARCHAR NOT NULL, lfdnr INTEGER NOT NULL)
Table: modulbt (mbtid INTEGER NOT NULL, modul_mid INTEGER NOT NULL, sws INTEGER NOT NULL, modulbtart_mbaid INTEGER

In [ ]:
con.close()

# Ops

In [21]:
ddbu.columns_print(con)

Table: belegung
  {'col': 'kursbt_kbtid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'studentin_prid', 'type': 'INTEGER', 'nullable': False}
Table: einzeltermin
  {'col': 'tid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'datumuhrzeitvon', 'type': 'DATE', 'nullable': False}
  {'col': 'datumuhrzeitbis', 'type': 'DATE', 'nullable': False}
Table: kurs
  {'col': 'kid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'modul_mid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'semester_sid', 'type': 'INTEGER', 'nullable': False}
Table: kursbt
  {'col': 'kbtid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'kurs_kid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'modulbt_mbtid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'raum_rid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'lehrperson_prid', 'type': 'INTEGER', 'nullable': False}
  {'col': 'termin_tid', 'type': 'INTEGER', 'nullable': False}
Table: lehrperson
  {'col': 'prid', 'type': 'INTEGER', 'nullable': False}


In [ ]:
ddbu.primary_keys(con)

{'belegung': ['kursbt_kbtid', 'studentin_prid'],
 'einzeltermin': ['tid'],
 'kurs': ['kid'],
 'kursbt': ['kbtid'],
 'lehrperson': ['prid'],
 'modul': ['mid'],
 'modulart': ['maid'],
 'modulbt': ['mbtid'],
 'modulbtart': ['mbaid'],
 'person': ['pid'],
 'personenrolle': ['prid'],
 'raum': ['rid'],
 'rolle': ['rid'],
 'semester': ['sid'],
 'sonstige': ['prid'],
 'studentin': ['prid'],
 'tag': ['tid'],
 'termin': ['tid'],
 'terminart': ['taid'],
 'wochentermin': ['tid'],
 'zeitblock': ['zbid']}

In [30]:
ddbu.foreign_keys(con)


{'belegung': [{'kursbt': [['kursbt_kbtid'], ['kbtid']]},
  {'studentin': [['studentin_prid'], ['prid']]}],
 'einzeltermin': [{'termin': [['tid'], ['tid']]}],
 'kurs': [{'modul': [['modul_mid'], ['mid']]},
  {'semester': [['semester_sid'], ['sid']]}],
 'kursbt': [{'kurs': [['kurs_kid'], ['kid']]},
  {'lehrperson': [['lehrperson_prid'], ['prid']]},
  {'modulbt': [['modulbt_mbtid'], ['mbtid']]},
  {'raum': [['raum_rid'], ['rid']]},
  {'termin': [['termin_tid'], ['tid']]}],
 'lehrperson': [{'personenrolle': [['prid'], ['prid']]}],
 'modul': [{'modulart': [['modulart_maid'], ['maid']]}],
 'modulbt': [{'modul': [['modul_mid'], ['mid']]},
  {'modulbtart': [['modulbtart_mbaid'], ['mbaid']]}],
 'personenrolle': [{'person': [['person_pid'], ['pid']]},
  {'rolle': [['rolle_rid'], ['rid']]}],
 'sonstige': [{'personenrolle': [['prid'], ['prid']]}],
 'studentin': [{'personenrolle': [['prid'], ['prid']]}],
 'termin': [{'terminart': [['terminart_taid'], ['taid']]}],
 'wochentermin': [{'tag': [['tag_ti

# YAML

In [31]:
schema_dict = ddbu.schema_yaml(con)
schema_dict

{'tables': [{'name': 'belegung',
   'columns': [{'col': 'kursbt_kbtid', 'type': 'INTEGER', 'nullable': False},
    {'col': 'studentin_prid', 'type': 'INTEGER', 'nullable': False}],
   'primary_key': ['kursbt_kbtid', 'studentin_prid'],
   'foreign_keys': [{'name': 'fk_belegung_kursbt_kursbt_kbtid',
     'columns': ['kursbt_kbtid'],
     'ref_table': 'kursbt',
     'ref_columns': ['kbtid']},
    {'name': 'fk_belegung_studentin_studentin_prid',
     'columns': ['studentin_prid'],
     'ref_table': 'studentin',
     'ref_columns': ['prid']}]},
  {'name': 'einzeltermin',
   'columns': [{'col': 'tid', 'type': 'INTEGER', 'nullable': False},
    {'col': 'datumuhrzeitvon', 'type': 'DATE', 'nullable': False},
    {'col': 'datumuhrzeitbis', 'type': 'DATE', 'nullable': False}],
   'primary_key': ['tid'],
   'foreign_keys': [{'name': 'fk_einzeltermin_termin_tid',
     'columns': ['tid'],
     'ref_table': 'termin',
     'ref_columns': ['tid']}]},
  {'name': 'kurs',
   'columns': [{'col': 'kid', 'ty

In [32]:
ddbu.schema_yaml_print(con)

tables:
- name: belegung
  columns:
  - col: kursbt_kbtid
    type: INTEGER
    nullable: false
  - col: studentin_prid
    type: INTEGER
    nullable: false
  primary_key:
  - kursbt_kbtid
  - studentin_prid
  foreign_keys:
  - name: fk_belegung_kursbt_kursbt_kbtid
    columns:
    - kursbt_kbtid
    ref_table: kursbt
    ref_columns:
    - kbtid
  - name: fk_belegung_studentin_studentin_prid
    columns:
    - studentin_prid
    ref_table: studentin
    ref_columns:
    - prid
- name: einzeltermin
  columns:
  - col: tid
    type: INTEGER
    nullable: false
  - col: datumuhrzeitvon
    type: DATE
    nullable: false
  - col: datumuhrzeitbis
    type: DATE
    nullable: false
  primary_key:
  - tid
  foreign_keys:
  - name: fk_einzeltermin_termin_tid
    columns:
    - tid
    ref_table: termin
    ref_columns:
    - tid
- name: kurs
  columns:
  - col: kid
    type: INTEGER
    nullable: false
  - col: modul_mid
    type: INTEGER
    nullable: false
  - col: semester_sid
    type: I